In [ ]:
# 2019/10/30
# Categorical Feature Encoding on Kaggle
# AUC: 0.80399 in Privacy test

In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/cat-in-the-dat/test.csv
/kaggle/input/cat-in-the-dat/train.csv
/kaggle/input/cat-in-the-dat/sample_submission.csv


In [12]:
catTrain = pd.read_csv("/kaggle/input/cat-in-the-dat/train.csv")
catTest = pd.read_csv("/kaggle/input/cat-in-the-dat/test.csv")
cat = pd.concat([catTrain, catTest], axis = 0, ignore_index = True, sort = False)
print(cat.shape) # number of rows and columns. (rows, cols)
print(cat.tail(5))

(500000, 25)
            id  bin_0  bin_1  bin_2 bin_3 bin_4  nom_0      nom_1    nom_2  \
499995  499995      0      0      0     F     N  Green     Square     Lion   
499996  499996      1      0      0     F     Y  Green  Trapezoid     Lion   
499997  499997      0      1      1     T     Y  Green  Trapezoid     Lion   
499998  499998      1      0      0     T     Y   Blue       Star  Hamster   
499999  499999      0      0      0     T     Y  Green       Star     Lion   

             nom_3  ...      nom_9 ord_0        ord_1        ord_2 ord_3  \
499995      Canada  ...  acc31291f     1       Novice     Lava Hot     j   
499996       China  ...  eae3446d0     1  Contributor     Lava Hot     f   
499997      Canada  ...  33dd3cf4b     1       Novice  Boiling Hot     g   
499998  Costa Rica  ...  d4cf587dd     2  Grandmaster  Boiling Hot     g   
499999       India  ...  2d610f52c     2       Novice     Freezing     l   

       ord_4  ord_5 day month target  
499995     A     Gb   

In [13]:
cat = cat.drop(columns = ["id", "nom_9", "target"])

In [14]:
# one hot encoding
onehot = pd.get_dummies(cat, columns = cat.columns, drop_first = True)
del cat

In [15]:
from scipy.sparse import csr_matrix
x_train = csr_matrix(onehot[:300000])
y_train = catTrain["target"][:300000]
x_test = csr_matrix(onehot[catTrain.shape[0]:])


In [16]:
from sklearn.linear_model import LogisticRegression
logModel = LogisticRegression()
logModel.fit(x_train, y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [17]:
# predict test data
predictions = logModel.predict_proba(x_test)
predict_df = pd.DataFrame(predictions)

predict_df.columns = ["0_prob", "target"]
out = pd.concat([catTest["id"], predict_df["target"]], axis = 1)
out.head(5)

,id,target
0,300000,0.395867
1,300001,0.680815
2,300002,0.126933
3,300003,0.512431
4,300004,0.874324


In [18]:
import os
os.getcwd()
out.to_csv("Result_all.csv", index=0)